[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ianjure/stock-market-trend-forecast/blob/main/Stock_Market_Trend_Forecast_Notebook.ipynb)

**Author:** Ian Jure Macalisang

**Email:** ianjuremacalisang2@gmail.com

**Link:** https://github.com/ianjure/stock-market-trend-forecast

**Website:** https://forecaststocktrend.streamlit.app/

# **I. Data Collection**

We will access the Yahoo Finance API using the yfinance library to fetch historical stock price data. All ticker symbols are found [here](https://stockanalysis.com/stocks/).

In [ ]:
import yfinance as yf
import pandas as pd
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
ticker = 'goog' # @param {type:"string"}
timeframe = 'Week' # @param ["Week", "Month"]

In [ ]:
# FETCH DATA
stock = yf.Ticker(ticker)
stock_name = stock.info['shortName']
stock_ticker = stock.info['symbol']
stock = stock.history(period="max")

# **II. Data Preprocessing**

In [ ]:
# CHECK IF THE STOCK IS PUBLICLY TRADED FOR MORE THAN 10 YEARS
if stock.shape[0] < 2470:
  print("The stock is not publicly traded for more than 10 years.")
else:
  print("The stock is publicly traded for more than 10 years.")

The stock is publicly traded for more than 4 years.


In [ ]:
def format_week(stock):

    # CLEANING
    del stock["Dividends"]
    del stock["Stock Splits"]
    stock.index = stock.index.strftime('%Y-%m-%d')
    stock.index = pd.to_datetime(stock.index)
    stock_date = pd.date_range(start="1990-01-01", end=stock.index[-1].tz_localize(None), freq="W-FRI")
    stock = stock.reindex(stock_date, method="ffill")

    # FEATURE ENGINEERING
    stock["Next Week"] = stock["Close"].shift(-1)
    stock["Target"] = (stock["Next Week"] > stock["Close"]).astype(int)

    horizons = [4,13,26,52,208]

    for horizon in horizons:
      rolling_averages = stock.rolling(horizon).mean()

      ratio_column = f"Close_Ratio_{horizon}"
      stock[ratio_column] = stock["Close"] / rolling_averages["Close"]

      trend_column = f"Trend_{horizon}"
      stock[trend_column] = stock.shift(1).rolling(horizon).sum()["Target"]

    stock = stock.dropna(subset=['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208'])

    # SCALING
    from sklearn.preprocessing import StandardScaler, MinMaxScaler

    features_to_scale = ["Open","High","Low","Close","Volume"]
    standard_scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()

    standard_scaled = standard_scaler.fit_transform(stock[features_to_scale])
    standard_scaled = pd.DataFrame(standard_scaled, index=stock.index, columns=['Standard Open', 'Standard High', 'Standard Low', 'Standard Close', 'Standard Volume'])

    minmax_scaled = minmax_scaler.fit_transform(stock[features_to_scale])
    minmax_scaled = pd.DataFrame(minmax_scaled, index=stock.index, columns=['MinMax Open', 'MinMax High', 'MinMax Low', 'MinMax Close', 'MinMax Volume'])

    stock = pd.concat([stock, standard_scaled], axis=1)
    stock = pd.concat([stock, minmax_scaled], axis=1)

    # ADDING FEATURES
    from fredapi import Fred

    API_KEY = '9405ac5ae5fa27cf2120c9a4a5b2c8f5'
    fred = Fred(api_key=API_KEY)

    crude_oil = fred.get_series('DCOILWTICO').rename('Price')
    crude_oil = crude_oil.reindex_like(stock)
    stock['Crude Oil'] = crude_oil
    stock['Crude Oil'] = crude_oil.ffill()
    stock = stock.dropna(subset=['Crude Oil'])

    effective_rate = fred.get_series('DFF').rename('Rate')
    effective_rate = effective_rate.reindex_like(stock)
    stock['Effective Rate'] = effective_rate
    stock['Effective Rate'] = effective_rate.ffill()

    interest_rate = fred.get_series('T10Y3M').rename('Rate')
    interest_rate = interest_rate.reindex_like(stock)
    stock['Interest Rate'] = interest_rate
    stock['Interest Rate'] = interest_rate.ffill()

    return stock

def format_month(stock):

    # CLEANING
    del stock["Dividends"]
    del stock["Stock Splits"]
    stock.index = stock.index.strftime('%Y-%m-%d')
    stock.index = pd.to_datetime(stock.index)
    stock_date = pd.date_range(start="1990-01-01", end=stock.index[-1].tz_localize(None), freq="M")
    stock = stock.reindex(stock_date, method="ffill")
    stock["Next Month"] = stock["Close"].shift(-1)
    stock["Target"] = (stock["Next Month"] > stock["Close"]).astype(int)

    # FEATURE ENGINEERING
    horizons = [2,6,12,30,60]

    for horizon in horizons:
      rolling_averages = stock.rolling(horizon).mean()

      ratio_column = f"Close_Ratio_{horizon}"
      stock[ratio_column] = stock["Close"] / rolling_averages["Close"]

      trend_column = f"Trend_{horizon}"
      stock[trend_column] = stock.shift(1).rolling(horizon).sum()["Target"]

    stock = stock.dropna(subset=['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60'])

    # SCALING
    from sklearn.preprocessing import StandardScaler, MinMaxScaler

    features_to_scale = ["Open","High","Low","Close","Volume"]
    standard_scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()

    standard_scaled = standard_scaler.fit_transform(stock[features_to_scale])
    standard_scaled = pd.DataFrame(standard_scaled, index=stock.index, columns=['Standard Open', 'Standard High', 'Standard Low', 'Standard Close', 'Standard Volume'])

    minmax_scaled = minmax_scaler.fit_transform(stock[features_to_scale])
    minmax_scaled = pd.DataFrame(minmax_scaled, index=stock.index, columns=['MinMax Open', 'MinMax High', 'MinMax Low', 'MinMax Close', 'MinMax Volume'])

    stock = pd.concat([stock, standard_scaled], axis=1)
    stock = pd.concat([stock, minmax_scaled], axis=1)

    # ADDING FEATURES
    from fredapi import Fred

    API_KEY = 'YOUR FRED API KEY HERE'
    fred = Fred(api_key=API_KEY)

    crude_oil = fred.get_series('DCOILWTICO').rename('Price')
    crude_oil = crude_oil.reindex_like(stock)
    stock['Crude Oil'] = crude_oil
    stock['Crude Oil'] = crude_oil.ffill()
    stock = stock.dropna(subset=['Crude Oil'])

    effective_rate = fred.get_series('DFF').rename('Rate')
    effective_rate = effective_rate.reindex_like(stock)
    stock['Effective Rate'] = effective_rate
    stock['Effective Rate'] = effective_rate.ffill()

    interest_rate = fred.get_series('T10Y3M').rename('Rate')
    interest_rate = interest_rate.reindex_like(stock)
    stock['Interest Rate'] = interest_rate
    stock['Interest Rate'] = interest_rate.ffill()

    return stock

In [ ]:
stock = format_week(stock) if timeframe == 'Week' else format_month(stock)
stock

,Open,High,Low,Close,Volume,Next Week,Target,Close_Ratio_4,Trend_4,Close_Ratio_13,Trend_13,Close_Ratio_26,Trend_26,Close_Ratio_52,Trend_52,Close_Ratio_208,Trend_208,Standard Open,Standard High,Standard Low,Standard Close,Standard Volume,MinMax Open,MinMax High,MinMax Low,MinMax Close,MinMax Volume,Crude Oil,Effective Rate,Interest Rate
2008-08-08,11.945332,12.333434,11.834374,12.315023,150132633.0,12.691682,1,1.022658,2.0,0.928527,6.0,0.964173,13.0,0.879053,30.0,1.227548,125.0,-0.951255,-0.944086,-0.951090,-0.942014,0.994724,0.029949,0.030351,0.031363,0.031204,0.251196,115.42,2.01,2.24
2008-08-15,12.613066,12.704370,12.575997,12.691682,142359606.0,12.205062,0,1.038473,3.0,0.966679,6.0,0.995116,13.0,0.905627,31.0,1.259061,126.0,-0.936333,-0.935889,-0.934349,-0.933610,0.896021,0.033643,0.032350,0.035457,0.033235,0.238118,113.46,2.08,1.99
2008-08-22,12.227701,12.311790,12.177447,12.205062,92232419.0,11.525883,0,0.999363,2.0,0.936994,6.0,0.958199,13.0,0.871630,30.0,1.205288,126.0,-0.944945,-0.944564,-0.943346,-0.944468,0.259504,0.031511,0.030235,0.033257,0.030611,0.153785,114.48,2.02,2.17
2008-08-29,11.686597,11.717944,11.502000,11.525883,154504960.0,11.052200,0,0.945953,2.0,0.901071,5.0,0.905414,13.0,0.824590,29.0,1.133354,126.0,-0.957037,-0.957687,-0.958593,-0.959621,1.050244,0.028518,0.027034,0.029528,0.026948,0.258552,115.55,1.94,2.11
2008-09-05,11.083049,11.256451,10.948457,11.052200,182051827.0,10.888252,0,0.931205,1.0,0.880204,5.0,0.867493,13.0,0.792739,28.0,1.082462,125.0,-0.970524,-0.967885,-0.971088,-0.970190,1.400035,0.025180,0.024547,0.026473,0.024394,0.304897,106.47,1.97,1.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-26,168.770004,169.839996,165.865005,168.679993,25150100.0,168.399994,0,0.928331,1.0,0.949082,7.0,1.031392,15.0,1.126127,31.0,1.383206,115.0,2.553224,2.536430,2.525828,2.546782,-0.592311,0.897394,0.879176,0.881574,0.874455,0.040926,78.58,5.33,-1.18
2024-08-02,168.190002,170.210007,166.389999,168.399994,18907800.0,165.389999,0,0.957839,0.0,0.947670,7.0,1.023656,15.0,1.118513,31.0,1.375825,114.0,2.540263,2.544607,2.537678,2.540535,-0.671576,0.894186,0.881170,0.884472,0.872945,0.030424,74.99,5.33,-1.49
2024-08-09,161.645004,165.520004,160.929993,165.389999,13549200.0,164.740005,0,0.970229,0.0,0.932631,6.0,1.001766,14.0,1.093580,30.0,1.346468,113.0,2.394005,2.440969,2.414430,2.473376,-0.739620,0.857984,0.855895,0.854334,0.856712,0.021408,78.28,5.33,-1.39
2024-08-16,163.410004,166.949997,163.080002,164.740005,16853100.0,167.429993,1,0.987635,0.0,0.933969,5.0,0.992479,14.0,1.084212,30.0,1.336687,112.0,2.433447,2.472568,2.462962,2.458873,-0.697667,0.867747,0.863601,0.866201,0.853207,0.026967,78.05,5.33,-1.44


In [ ]:
# CHECK FOR NULL VALUES
print(stock.isnull().sum())

# CHECK THE NUMBER OF ROWS AND COLUMNS
print(stock.shape)

Open               0
High               0
Low                0
Close              0
Volume             0
Next Week          1
Target             0
Close_Ratio_4      0
Trend_4            0
Close_Ratio_13     0
Trend_13           0
Close_Ratio_26     0
Trend_26           0
Close_Ratio_52     0
Trend_52           0
Close_Ratio_208    0
Trend_208          0
Standard Open      0
Standard High      0
Standard Low       0
Standard Close     0
Standard Volume    0
MinMax Open        0
MinMax High        0
MinMax Low         0
MinMax Close       0
MinMax Volume      0
Crude Oil          0
Effective Rate     0
Interest Rate      0
dtype: int64
(838, 30)


# **III. Model Training**

In [ ]:
"""
BACKTESTING FUNCTION
"""

def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    conf = preds.copy()
    conf = pd.Series(conf, index=test.index, name="Confidence")
    preds[preds >= .6] = 1
    preds[preds < .6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    combined = pd.concat([combined, conf], axis=1)
    return combined

def backtest(data, model, predictors, timeframe):
    all_predictions = []
    if timeframe == 'Week':
        for i in range(520, data.shape[0], 52):
            train = data.iloc[0:i].copy()
            test = data.iloc[i:(i+52)].copy()
            predictions = predict(train, test, predictors, model)
            all_predictions.append(predictions)
    else:
        for i in range(120, data.shape[0], 12):
            train = data.iloc[0:i].copy()
            test = data.iloc[i:(i+12)].copy()
            predictions = predict(train, test, predictors, model)
            all_predictions.append(predictions)
    combined = pd.concat(all_predictions)
    return combined

In [ ]:
# IMPORTING THE MODELS AND TOOLS
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score

In [ ]:
# INITIALIZE PREDICTORS
if timeframe == 'Week':
    predictors = {
        'Original': ['Open', 'High', 'Low', 'Close', 'Volume']	,
        'Standard Scaled': ['Standard Open', 'Standard High', 'Standard Low', 'Standard Close', 'Standard Volume'],
        'MinMax Scaled': ['MinMax Open', 'MinMax High', 'MinMax Low', 'MinMax Close', 'MinMax Volume'],
        'Ratio Trend': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208'],
        'Economic Indicators': ['Crude Oil', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Economic Indicators': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Crude Oil', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Interest Rate': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Interest Rate'],
        'Ratio Trend Crude Oil Rate': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Crude Oil'],
        'Ratio Trend Effective Rate': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Effective Rate'],
        'Ratio Trend Effective Interest': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Crude Oil Interest': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Crude Oil', 'Interest Rate'],
        'Ratio Trend Crude Oil Effective': ['Close_Ratio_4', 'Trend_4', 'Close_Ratio_13', 'Trend_13', 'Close_Ratio_26', 'Trend_26', 'Close_Ratio_52', 'Trend_52', 'Close_Ratio_208', 'Trend_208', 'Crude Oil', 'Effective Rate']
    }
else:
    predictors = {
        'Original': ['Open', 'High', 'Low', 'Close', 'Volume']	,
        'Standard Scaled': ['Standard Open', 'Standard High', 'Standard Low', 'Standard Close', 'Standard Volume'],
        'MinMax Scaled': ['MinMax Open', 'MinMax High', 'MinMax Low', 'MinMax Close', 'MinMax Volume'],
        'Ratio Trend': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60'],
        'Economic Indicators': ['Crude Oil', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Economic Indicators': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Crude Oil', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Interest Rate': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Interest Rate'],
        'Ratio Trend Crude Oil Rate': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Crude Oil'],
        'Ratio Trend Effective Rate': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Effective Rate'],
        'Ratio Trend Effective Interest': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Effective Rate', 'Interest Rate'],
        'Ratio Trend Crude Oil Interest': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Crude Oil', 'Interest Rate'],
        'Ratio Trend Crude Oil Effective': ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_6', 'Trend_6', 'Close_Ratio_12', 'Trend_12', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Crude Oil', 'Effective Rate']
    }

# INITIALIZE CLASSIFIERS
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=1),
    'KNN': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(random_state=1),
    'Gradient Boosting': GradientBoostingClassifier(random_state=1)
}

In [ ]:
"""
TRAINING ALL MODELS WITH ALL PREDICTORS
"""

# SPLITTING THE DATASET
train = stock.iloc[:-100] # get all the data except the last 100 rows as the training set
test = stock.iloc[-100:] # get the last 100 rows as the testing set

top_results = {'Classifier': [],
               'Predictors': [],
               'Accuracy': [],
               'Precision': [],
               'Total Errors': []
               }

# ITERATE OVER PREDICTORS
for predictor_name, prd in predictors.items():

    # DICTIONARY TO STORE INITIAL RESULTS
    results = {'Classifier': [],
               'Predictors': [],
               'Accuracy': [],
               'Precision': [],
               'Total Errors': []
               }

    # ITERATE OVER CLASSIFIERS
    for clf_name, clf in classifiers.items():

        # FIT THE DATA TO THE MODEL AND BACKTEST
        preds = backtest(stock, clf, prd, timeframe)

        # CALCULATE PRECISION AND ERRORS
        accuracy = accuracy_score(preds["Target"], preds["Predictions"])
        precision = precision_score(preds["Target"], preds["Predictions"])
        errors = (preds["Target"] != preds["Predictions"]).sum()

        # STORE RESULTS
        results['Classifier'].append(clf_name)
        results['Predictors'].append(predictor_name)
        results['Accuracy'].append(accuracy)
        results['Precision'].append(precision)
        results['Total Errors'].append(errors)

    # CREATE DATAFRAME FROM RESULTS
    results_df = pd.DataFrame(results).sort_values(by='Accuracy', ascending=False).reset_index(drop=True)

    # APPEND TO TOP RESULT
    top_results['Classifier'].append(results['Classifier'][0])
    top_results['Predictors'].append(results['Predictors'][0])
    top_results['Accuracy'].append(results['Accuracy'][0])
    top_results['Precision'].append(results['Precision'][0])
    top_results['Total Errors'].append(results['Total Errors'][0])

# DISPLAY THE TOP RESULTS
top_results_df = pd.DataFrame(top_results).sort_values(by='Accuracy', ascending=False).reset_index(drop=True)
top_results_df[['Classifier', 'Predictors', 'Accuracy', 'Precision', 'Total Errors']]

,Classifier,Predictors,Accuracy,Precision,Total Errors
0,Logistic Regression,Ratio Trend Economic Indicators,0.537736,0.588235,147
1,Logistic Regression,Ratio Trend Effective Interest,0.531447,0.570513,149
2,Logistic Regression,Ratio Trend Crude Oil Effective,0.525157,0.592593,151
3,Logistic Regression,Economic Indicators,0.522013,0.579832,152
4,Logistic Regression,Ratio Trend Crude Oil Interest,0.503145,0.552000,158
5,Logistic Regression,Ratio Trend Interest Rate,0.496855,0.534161,160
6,Logistic Regression,Ratio Trend Crude Oil Rate,0.493711,0.575758,161
7,Logistic Regression,Ratio Trend Effective Rate,0.474843,0.535714,167
8,Logistic Regression,Original,0.462264,0.000000,171
9,Logistic Regression,Ratio Trend,0.462264,0.500000,171


In [ ]:
# TRAINING THE FINAL MODEL WITH THE WHOLE DATASET
final_model = classifiers[top_results_df['Classifier'][0]]

# **IV. Model Testing**

In [ ]:
result = backtest(stock, final_model, predictors[top_results_df['Predictors'][0]], timeframe)

In [ ]:
result

,Target,Predictions,Confidence
2018-07-27,0,0.0,0.557077
2018-08-03,1,1.0,0.604439
2018-08-10,0,1.0,0.619736
2018-08-17,1,1.0,0.603223
2018-08-24,0,1.0,0.608580
...,...,...,...
2024-07-26,0,1.0,0.606692
2024-08-02,0,1.0,0.676638
2024-08-09,0,1.0,0.665850
2024-08-16,1,1.0,0.663860


In [ ]:
if timeframe == 'Week':
    if result['Predictions'][-1] > 0:
        print(f"{stock_name} ({stock_ticker}) stock price will go up next week.")
        print(f"Confidence: {round(result['Confidence'][-1] * 100)}%")
    else:
        print(f"{stock_name} ({stock_ticker}) stock price will go down next week.")
        print(f"Confidence: {round((1 - result['Confidence'][-1]) * 100)}%")
else:
    if result['Predictions'][-1] > 0:
        print(f"{stock_name} ({stock_ticker}) stock price will go up next month.")
        print(f"Confidence: {round(result['Confidence'][-1] * 100)}%")
    else:
        print(f"{stock_name} ({stock_ticker}) stock price will go down next month.")
        print(f"Confidence: {round((1 - result['Confidence'][-1]) * 100)}%")

Alphabet Inc. (GOOG) stock price will go up next week.
Confidence: 62%
